In [1]:
import numpy as np
import pandas as pd
import sima2py as sapy

In [6]:
#this is great, but I need to be able to get more events, that means analyzing/skimming multiple files
import re

fileregex = re.compile('Run68_gdirect_bknd_R68_PuBe_0x0006_1M_15507\\S+.txt')
dirpath,f = sapy.listFiles('/data/chocula/villaa/k100Sim_Data/captureCal/',fileregex)

['Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550748424_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550723674_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550713990_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550779886_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550722626_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550719315_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550765618_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550759060_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550732394_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550767758_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550723948_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550784888_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550735789_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550793433_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_1550750159_000_000.txt', 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_

In [7]:
print(np.size(f))

142


# First Skim of this Data

For the first skim of this data I used the command:

```./sima2py.py --regex 'Run68_gdirect_bknd_R68_PuBe_0x0006_1M_15507\S+.txt' --filedir /data/chocula/villaa/k100Sim_Data/captureCal/ --outfile /data/chocula/villaa/k100Sim_Data/captureCalhdf5/R68_gdirect_testskim.h5```

This created a file `/data/chocula/villaa/k100Sim_Data/captureCalhdf5/R68_gdirect_testskim.h5` which is about 229Mb and contains approximately 142 live-seconds of data. I need to practice the first skim on this, and then generalize to the larger data sets. 

In [8]:
#play around with some hits data stored in h5 file
#===============to suppress h5py warning see:
#https://github.com/h5py/h5py/issues/961
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h5py
warnings.resetwarnings()
f = h5py.File("/data/chocula/villaa/k100Sim_Data/captureCalhdf5/R68_gdirect_testskim.h5","r")

for i in f:
    print(i)

hits = f['geant4/hits']

geant4


In [9]:
#using the examples above let's get byzantine and make a bad-ass data frame.

data=hits
#first do some cuts:
#first some hit-level cuts
cHVDet = np.zeros(np.shape(data)[0],dtype=bool)
cZeroEdep = np.zeros(np.shape(data)[0],dtype=bool)
cNeutron = np.zeros(np.shape(data)[0],dtype=bool)
cGamma = np.zeros(np.shape(data)[0],dtype=bool)
cNR = np.zeros(np.shape(data)[0],dtype=bool)

cHVDet[data[:,1]==1] = True
cZeroEdep[data[:,6]==0] = True
cNeutron[data[:,4]==2112] = True
cGamma[data[:,4]==22] = True
cNR[data[:,4]>3000] = True


#now some event-level cuts
#evWithHVhits = newev[cHVDet & ~cZeroEdep]
#cWithHVHits = np.isin(newev,evWithHVhits)
#print(np.sum(cWithHVHits))
#print(np.shape(np.unique(evWithHVhits)))


#now make a dataframe with the restricted data
nr_data = data[:,[0,4,5,6,21]]
nr_data = nr_data[cHVDet&~cZeroEdep&cNR,:]
print(np.shape(nr_data))
nr_dataframe = pd.DataFrame(data=nr_data)

#print(nr_dataframe)

#print(np.max(nr_dataframe.groupby([0,1],axis=0).size()))
max_vec = np.max(nr_dataframe.groupby([0,1],axis=0).size())

evec = np.zeros((0,max_vec))
nhit = np.zeros((0,1))

for i in nr_dataframe.groupby([0,1],axis=0)[3].apply(list):
    #print(i)
    #print(np.shape(i))
    vector = np.zeros((1,max_vec))
    #print(np.shape(vector[0,0:np.shape(i)[0]]))
    vector[0,0:np.shape(i)[0]] = np.transpose(np.asarray(i))
    evec = np.append(evec,vector,0)
    nhit = np.append(nhit,np.shape(i)[0])
    
print(np.shape(evec))
print(np.shape(nhit))
#print(max_vec)
#print(np.sum(evec,1))
print(np.sum(nhit[nhit==1]))

(424, 5)
(301, 8)
(301,)
228.0
